In [2]:
import numpy as np
import os

In [9]:
a = np.load('inputs/train_images_20190914/img_0011165.npy')
b = np.load('inputs/train_images_mask_20190914/msk_0011165.npy')

In [14]:
np.reshape(a, [1, 525, 325]).shape

(1, 525, 325)

In [16]:
[1, a.shape[0], a.shape[1]]

[1, 525, 325]

3